In [167]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [168]:
# To make an enviroment we need:
#    - Init
#    - Reset
#    - State Space
#    - State Space Plus
#    - Way to denote possible actions
#    - Way to make sure move is legal
#    - Way to actually affect the env
#    - Step func needs to return new position, reward, isTerminal?, debug info
#    - Reset
#    - Print env to terminal

In [214]:
class GridWorld(object):
    
    def __init__(self, m, n, loot):
        
        self.grid = np.zeros((m, n))
        self.m    = m
        self.n    = n
        
        self.stateSpace = [i for i in range(self.m * self.n)]
        
        self.stateSpace.remove(self.m * self.n -1)
        
        self.stateSpacePlus = [i for i in range(self.m * self.n)]
        
        self.guardSpace = [74,75,76,77,78,79]
        #self.guardSpace = [14, 15, 16, 17]
            
        self.actionSpace = {
                            'U': -self.m,
                            'D': self.m,
                            'L': -1,
                            'R': 1
                           }
        
        self.guardActionSpace = {
                             #    'U': -self.m,
                           #      'D': self.m,
                                 'L': -1,
                                 'R': 1
                                }
        
        self.possibleActions = ['U', 'D', 'L', 'R']
        
        self.SecGuardActions = ['L', 'R']
        
        self.addLoot(loot)
        
        self.addWalls()
        
        self.agentPosition    = 0
        self.secGuardPosition = 75
        self.exit             = 5
        
        self.grid[self.m-1][self.n-1] = 5
        
        self.FLAGS = 0
        
    def addLoot(self, loot):
        
        self.loot = loot
        
        i = 2
        
        for square in self.loot:
            
            # Find what position we are in
            x = square // self.m
            y = square %  self.n

            self.grid[x][y] = i
            self.grid
            
    
    def addWalls(self):
        
        #self.WallSpace = [30, 31, 32, 39, 48, 49, 50]
        self.WallSpace = [3,4,5,7,8,9,15,17,19,21,27,29,31,33]
        
        j = 3            
        
        for square in self.WallSpace:
            
            x = square // self.m
            y = square %  self.n 
            
            self.grid[x][y] = j
        
    def isTerminalState(self, state):
        
        global reward, epRewards
        
        if self.FLAGS == 0:
            epRewards -= 1
            
        return state in self.stateSpacePlus and state not in self.stateSpace
    
    def isGuardTerminalState(self, guardState):
        
        global done

        x, y = self.getAgentRowAndColumn()
    
        print("[!] Agent at {}, {}".format(x, y))
        
        if guardState == self.agentPosition or self.secGuardPosition == self.agentPosition:
            
            print("Landed on same tile, quitting")
            done = True
        
        return guardState
    
    def getAgentRowAndColumn(self):
        
        x = self.agentPosition // self.m
        y = self.agentPosition %  self.n
        return x,y
    
    def getSecurityGuardRowAndColumn(self):
        
        x = self.secGuardPosition // self.m
        y = self.secGuardPosition %  self.n
        return x, y
    
    def setState(self, state):
        
        x, y               = self.getAgentRowAndColumn()
        self.grid[x][y]    = 0
        self.agentPosition = state
        x, y               = self.getAgentRowAndColumn()
        self.grid[x][y]    = 1
        
    def setGuardState(self, guardState):
        
        x, y                  = self.getSecurityGuardRowAndColumn()
        self.grid[x][y]       = 0
        self.secGuardPosition = guardState
        x, y                  = self.getSecurityGuardRowAndColumn()
        self.grid[x][y]       = 4
        
    def offGridMove(self, newState, oldState):
        
        if newState not in self.stateSpacePlus:
            return True
        
        elif oldState % self.m == 0 and newState % self.m == self.m - 1:
            return True
        
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        
        else:
            return False


    def offGridGuardMove(self, newState, oldState):
        
        if newState not in self.guardSpace:
            return True
        
        elif oldState % self.m == 0 and newState % self.m == self.m - 1:
            return True
        
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        
        else:
            return False        
    
    def step(self, action):
        
        
        #env.render()
      
        resultState = self.agentPosition + self.actionSpace[action]
                
        print("[+] Agent Result State: {}".format(resultState))
        
        reward = -1 if not self.isTerminalState(resultState) else 0
        
        # If treasure tile
        if resultState in self.loot and self.FLAGS != 1:
            print("[+] Agent received the treasure. +50 ")
            reward += 50
            print("[+] Current Reward: {0}, Total Reward: {1}".format(reward, epRewards))
            self.FLAGS = 1
        
        if not self.offGridMove(resultState, self.agentPosition):
            if resultState in self.WallSpace:
                print("[+] Hit a wall..\n")
                return self.agentPosition, reward, self.isTerminalState(self.agentPosition), None
            else:
                print("[+] No issues, continuing game...")
                self.setState(resultState)
                return resultState, reward, self.isTerminalState(self.agentPosition), None
        else:
            print("[+] Attempted off-grid-move...")
            return self.agentPosition, reward, self.isTerminalState(self.agentPosition), None
         
    
    def Guardstep(self, action):
        
        guardState  = self.secGuardPosition + self.guardActionSpace[action]
        
        print("[+] Guard Result State: {}".format(guardState))
        
        reward = 0
        
        if not self.offGridGuardMove(guardState, self.secGuardPosition):
            print("[+] Guard No issues, continuing game...")
            self.setGuardState(guardState)
            return guardState, reward, self.isGuardTerminalState(self.secGuardPosition), None
        else:
            print("[+] Guard Attempted off-grid-move...")
            return self.secGuardPosition, reward, self.isGuardTerminalState(self.secGuardPosition), None

        
    def actionSpaceSample(self):
        return np.random.choice(self.possibleActions)
        
    def actionSpaceGuardSample(self):
        return np.random.choice(self.SecGuardActions)  
        
    def reset(self):
        
        self.grid = np.zeros((self.m, self.n))
        
        self.agentPosition            = 0
        #self.secGuardPosition         = 14
        self.secGuardPosition         = 75
        self.grid[self.m-1][self.m-1] = 5
        #self.loot = {40}
        self.loot = {16, 20}
        self.addLoot(self.loot)
        self.addWalls()
        
        return self.agentPosition

    def greset(self):
        
        self.secGuardPosition         = 75
        
        return self.secGuardPosition

        
#     def render(self):
#         print("-------------------------------------")
        
#         for row in self.grid:
               
#             for col in row:
#                # empty square
#                 if col == 0:
#                     print('-', end = '\t')
#                 # Agent
#                 elif col == 1:
#                     print('X', end = '\t')
#                # Entrance to Magic Square
#                 elif col == 2:
#                     print('Loot', end = '\t')
#                 # Return to Magic Square
#                 elif col == 3:
#                     print('W', end = '\t')
#                 # 
#                 elif col == 4:
#                     print('G', end = '\t')
#                 #
#                 elif col == 5:
#                     print('Out', end = '\t')

            
#             print('\n')
        
#         print("-------------------------------------");
        
    def render(self, seq_num):
        
        data = np.zeros((1200, 1200, 3), 
                         dtype = np.uint8)
        
        #print(self.grid)
      
        
        for i in range(12):
            
            for j in range(12):
                
                if self.grid[i,j] == 0:
                    data[i*100:i*100+100, j*100:j*100+100] = [0,0,0]
                    
                elif self.grid[i,j] == 1:
                    data[i*100:i*100+100, j*100:j*100+100] = [0,255,255]
                    
                elif self.grid[i,j] == 2:
                    data[i*100:i*100+100, j*100:j*100+100] = [255,0,255]
                    
                elif self.grid[i,j] == 3:
                    data[i*100:i*100+100, j*100:j*100+100] = [128,128,128]
                    
                elif self.grid[i,j] == 4:
                    data[i*100:i*100+100, j*100:j*100+100] = [255,204,204]
                    
                elif self.grid[i,j] == 5:
                    data[i*100:i*100+100, j*100:j*100+100] = [255,0,0]
                    
        img = Image.fromarray(data, 'RGB')
        img.save(f'images/Guard/cbs_{seq_num}.png') 
        

def maxAction(Q, state, actions):
    
    values  = np.array([Q[state,a] for a in actions])
    action  = np.argmax(values)
    
    return actions[action]


def maxGuardAction(W, Guardstate, Guardactions):
    
    values      = np.array([W[Guardstate, a] for a in Guardactions])
    Guardaction = np.argmax(values)
    
    return Guardactions[Guardaction]

In [215]:
if __name__ ==  '__main__':
    
    loot   = {16, 20}
    env    = GridWorld(12, 12, loot)
     
    # Hyperparameters
    
    ALPHA   = 0.1 # Learning Rate
    GAMMA   = 1.0 # Far Sighted
    EPSILON = 1.0 # Epsilon greedy 
    
    LOOTCOUNT = 0
    seq_num   = 0
    
    # Q Table
    
    Q = {}
    W = {}
    
    for state in env.stateSpacePlus:
        
        for action in env.possibleActions:
            
            # 0 is "Optimistic Initial Values" - Since agent recv reward if -1 per step, 
            # it can never have reward of 0, so it promotes exploration 
            
            Q[state, action] = 0

      
    for state in env.guardSpace:
        
        for action in env.SecGuardActions:
            
            W[state, action] = 0    
    
    # Params
    
    numGames     = 3
    totalRewards = np.zeros(numGames)
    
    for i in range(numGames):
        
        if i % 50 == 0:
            print("[+] Starting Game:", i)
        
        done = False
        
        epRewards    = 0
        gepRewards   = 0
        
        observation  = env.reset()
        gobservation = env.greset()
        
        while not done:
            
            seq_num += 1
            # Random number for epsilon greedy selection
            rand       = np.random.random()
            rand_guard = np.random.random()
            
            # find max action for given state
            action = maxAction(Q, observation, env.possibleActions) if rand < (1 - EPSILON) \
                            else env.actionSpaceSample()
        
            guardAction = maxGuardAction(W, gobservation, env.SecGuardActions) if rand_guard < (1 - EPSILON) \
                            else env.actionSpaceGuardSample()
            
            
            observation_, reward, done, info     = env.step(action)
#             print(env.__dict__)
#             print(guardAction)
            gobservation_, greward, gdone, ginfo = env.Guardstep(guardAction)

            epRewards  += reward
            gepRewards += greward
            
            
            action_  = maxAction(Q, observation_, env.possibleActions)
            
            gaction_ = maxGuardAction(W, gobservation_, env.SecGuardActions)
            
            Q[observation, action] = Q[observation, action] + \
                                     ALPHA * (reward + GAMMA * Q[observation_, action_] - Q[observation, action])
            
      #      print(Q)
      ###      print()
        #     print(W)
            
            W[observation, guardAction] = W[gobservation, guardAction] + \
                                     ALPHA * (greward + GAMMA * W[gobservation_, gaction_] - W[gobservation, guardAction])

            # Let environment know it has changed states
            observation = observation_ 
            gobservation = gobservation_
            
            env.render(seq_num)
            
        if EPSILON - 2 / numGames > 0:
            EPSILON -= 2/ numGames
            
        else:
            EPSILON = 0
            
        totalRewards[i] = epRewards    
        
    print("""
        ---------------------------------
        ---------------------------------
        ---------- JEWEL THIEF ----------
        ---------------------------------
        ---------------------------------
        - Number of Games: {0}
        - Number of Loot Found: {1}
        - 
    
    """.format(numGames, LOOTCOUNT))

[+] Starting Game: 0
[+] Agent Result State: 12
[+] No issues, continuing game...
[+] Guard Result State: 74
[+] Guard No issues, continuing game...
[!] Agent at 1, 0
[+] Agent Result State: 0
[+] No issues, continuing game...
[+] Guard Result State: 75
[+] Guard No issues, continuing game...
[!] Agent at 0, 0
[+] Agent Result State: -12
[+] Attempted off-grid-move...
[+] Guard Result State: 76
[+] Guard No issues, continuing game...
[!] Agent at 0, 0
[+] Agent Result State: -12
[+] Attempted off-grid-move...
[+] Guard Result State: 77
[+] Guard No issues, continuing game...
[!] Agent at 0, 0
[+] Agent Result State: -1
[+] Attempted off-grid-move...
[+] Guard Result State: 78
[+] Guard No issues, continuing game...
[!] Agent at 0, 0
[+] Agent Result State: 1
[+] No issues, continuing game...
[+] Guard Result State: 77
[+] Guard No issues, continuing game...
[!] Agent at 0, 1
[+] Agent Result State: 2
[+] No issues, continuing game...
[+] Guard Result State: 76
[+] Guard No issues, cont

In [ ]:

#####################
# End of Simulation #
#####################

In [ ]:
# Debug

In [103]:
env.isGuardTerminalState(15)

[!] Agent at 0, 0


15

In [51]:
loot   = {40}
env    = GridWorld(9, 9, loot)
env.render()
    
# Hyperparameters

ALPHA   = 0.1 # Learning Rate
GAMMA   = 1.0 # Far Sighted
EPSILON = 1.0 # Epsilon greedy 

LOOTCOUNT = 0
seq_num   = 0

# Q Table

Q = {}
W = {}

for state in env.stateSpacePlus:

    for action in env.possibleActions:

        # 0 is "Optimistic Initial Values" - Since agent recv reward if -1 per step, 
        # it can never have reward of 0, so it promotes exploration 

        Q[state, action] = 0


for state in env.guardSpace:

    for action in env.SecGuardActions:

        W[state, action] = 0    

# Params

numGames     = 1
totalRewards = np.zeros(numGames)

-------------------------------------
-	-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	-	

-	-	-	W	W	W	-	-	-	

-	-	-	W	Loot	-	-	-	-	

-	-	-	W	W	W	-	-	-	

-	-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	Out	

-------------------------------------


In [52]:
env.isGuardTerminalState(15)

[!] Agent at 0, 0


15

In [53]:
env.getSecurityGuardRowAndColumn()

(1, 5)

In [66]:
env.setGuardState(1)

In [67]:
env.getSecurityGuardRowAndColumn()

(0, 1)

In [75]:
env.isGuardTerminalState(0)

[!] Agent at 0, 0
Landed on same tile, quitting


0

In [114]:
env.getSecurityGuardRowAndColumn()

(1, 5)

In [115]:
print(env.secGuardPosition)

14


In [116]:
env.guardActionSpace

{'L': -1, 'R': 1}

In [117]:
guardState = env.secGuardPosition + env.guardActionSpace['R']

In [118]:
guardState

15

In [119]:
env.offGridGuardMove(guardState, env.secGuardPosition)

False

In [120]:
env.setGuardState(guardState)

In [121]:
env.getSecurityGuardRowAndColumn()

(1, 6)

In [126]:
txt = env.isGuardTerminalState(env.secGuardPosition)

[!] Agent at 0, 0


In [127]:
txt

15

In [177]:
h = 12
w = 12

In [179]:
j = np.zeros((h, w))

In [190]:
j.size

144